# Titanic with neuronal networks

The following code was given from the lecturer and should be expanded to create a neuronal network for the titanic dataset.

In [ ]:
import tensorflow as tf
import tensorflow.keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(..., activation='sigmoid', batch_input_shape=(None, 4))) #We have 4 input features
#...
model.add(Dense(1, activation='sigmoid'))
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

## Imports

Import the following packages:

In [ ]:
import tensorflow as tf
import tensorflow.keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import Dropout

import pandas as pd

import numpy as np

# Nice function to split a known data set to a test and trin set
from sklearn.model_selection import train_test_split

# Nice function for plotting
# Install via pip: pip install tensorflow-history-plot
from tensorflow_history_plot import show_acc

***

Read the data from the csv file and show some infos about the data:

In [ ]:
train_data = pd.read_csv("data/train.csv")

In [ ]:
train_data.info()

## Data preparation

Define the features we want to use to train our neuronal network

In [ ]:
# Read data again in case you have already manipulated it
train_data = pd.read_csv("data/train.csv") 

# Fill missing age in train data
train_data["Age"].fillna(train_data["Age"].median(skipna=True), inplace=True) 

# Define features we want to use
features = ["Pclass", "Survived", "Sex", "Age"]

# Get the features we want to use
train_data = pd.get_dummies(train_data[features])

Show the info of the modified dataset (notice that there should be a equal amount of "not null" values for the columns)

In [ ]:
train_data.info()

Split the data into training and test data. Therefore we will first split the data into a "X" and "y" part. The "X" part contains all features and the "y" part contains the label (survived or not). After that we will split the data into training and test data.

In [ ]:
X = train_data.drop('Survived', axis=1, inplace=False)
y = train_data['Survived']

print(f'Input features shape: {X.shape}')
print(f'Labels shape: {y.shape}')

Using the `train_test_split` function from the `sklearn.model_selection` package, split the data into training and test data. 
This is an easy and fast way to split the data into training and test data. The function will return 4 values: `X_train`, `X_test`, `y_train` and `y_test`. The `X` values are the features (in our case the columns `Pclass`, `Sex`, `Age`) and the `y` values are the labels (in our case the column `Survived`).

In [ ]:

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

## Neural Network

In the following section, the neuronal network is created and trained.

First we have to convert the data into a format that can be used by the neuronal network. Therefore we will use the `asarrray` function from the `numpy` package. This function will convert the data into a numpy array.

In [ ]:
X_train = np.asarray(X_train).astype('float32')
X_val = np.asarray(X_val).astype('float32')
y_train = np.asarray(y_train).astype('float32')
y_val = np.asarray(y_val).astype('float32')

The `X` values will be converted into a 2 dimensional array...

In [ ]:
X_train

... and the `y` values will be converted into a 1 dimensional array.

In [ ]:
y_train

***

Now we will define our neuronal network. Therefore we will use the `Sequential` class from the `keras.models` package. This class will create a new neuronal network. The `Sequential` class will take a list of layers as parameter.

In [ ]:
model = Sequential()

After defining our model we will add the layers to our model. Therefore we will use the `add` function. The first layer will be a `Dense` layer with the `sigmoid` activation function. The `Dense` layer will take the number of neurons as first parameter. The second parameter is the activation function. The `input_shape` parameter defines the input shape of the first layer. The input shape is the number of features. In our case the input shape is 4, because we have 4 features.

The last layer will only have one neuron, because we only want to predict one value (survived or not). The activation function will be the `sigmoid` function.

After defining our layers we will set the learning rate of the optimizer and compile the model. Therefore we will use the `compile` function.

In [ ]:
model.add(Dense(125, activation='sigmoid', batch_input_shape=(None, 4), name="Layer_1")) #We have 4 input features
model.add(Dense(500, activation='sigmoid', name="Layer_2"))
model.add(Dense(750, activation='sigmoid', name="Layer_3"))
model.add(Dense(1, activation='sigmoid', name="Layer_4"))

# Ask lecturer why this is not possible???
#model.add(Dense(1, activation="softmax", name="Layer_5"))

opt = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=opt,
              metrics=['accuracy'])

model.summary()

Finally we will train our model. Therefore we will use the `fit` function. The `fit` function will take the training data as first parameter, the labels as second parameter, the validation data as third parameter and the number of epochs as fourth parameter. The `fit` function will return a history object. This object contains the loss and accuracy of the training and validation data for each epoch.

In [ ]:
#history = model.fit(x, y, epochs=100, batch_size=32, validation_split=0.2, verbose=0)
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=50,
                    batch_size=32)

The accuracy can be shwon as simple values...

In [ ]:
model.evaluate(X_val, y_val)

... or as a nice plot using the `show_acc` function.

In [ ]:
show_acc.plot(history)

## Tests

Now we are ready to predict some values using the given test data set and the trained model.

In [ ]:
test_data = pd.read_csv("data/test.csv")

In [ ]:
test_data.info()

The data preparation is the same as before. We will convert the data into a numpy array and split the data into features and labels.

Notice that the "Survived" column is missing in the test data set. This is because we want to predict the "Survived" column now.

In [ ]:
# Read data again in case you have already manipulated it
test_data = pd.read_csv("data/test.csv")

# Fill missing age in train data
test_data["Age"].fillna(train_data["Age"].median(skipna=True), inplace=True) 

# Define features we want to use (again)
features = ['Pclass', 'Sex', 'Age']

##test_data.drop(['Name', 'Ticket', 'PassengerId', 'Cabin', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], axis=1, inplace=True)

test_data = pd.get_dummies(test_data[features])

test_data.info()

After converting the data into a numpy array we will predict the values using the `predict` function from the model. The `predict` function will take the features as parameter and will return the predicted values.

In [ ]:
test_data = np.asarray(test_data).astype('float32')

predictions = model.predict(test_data)
predictions

Because the values are not boolean values (what is needed because unless you are a zombie you are either dead or not, nothing between) we will convert the values into boolean values. Therefore we will use the `round` function from the `numpy` package.



In [ ]:
predictions = np.where(predictions > 0.5, 1, 0)
predictions = predictions.flatten()
predictions

## Export

Finally we will export the predicted values into a csv file. Therefore we will use the `to_csv` function from the `pandas` package.

In [ ]:
import os
import time
from datetime import datetime

timestamp = time.time()
timestamp = datetime.fromtimestamp(timestamp).strftime("%H_%M_%S")
print(timestamp)

test_data = pd.read_csv("data/test.csv")

predictions #Enhält die Predictions 0 für Tod 1 für Survived
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})

filename = 'my_submission_nn_' + timestamp + '.csv'
fileDir = os.path.join("export", filename)

output.to_csv(fileDir, index=False)
print("Your submission was successfully saved!")
print(fileDir)


### Ignore stuff below

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_data["Survived"], test_size=0.2, random_state=42)

y = train_data["Survived"]

features = ["Pclass"]
x = pd.get_dummies(train_data[features])

#...
#model.add(BatchNormalization())
#model.add(Dropout(0.2))
#model.add(Dense(10, activation='sigmoid'))
#model.add(keras.layers.normalization.BatchNormalization())
#model.add(Dropout(0.2))

In [ ]:

#model.add(Dense(1, activation='sigmoid', batch_input_shape=(None, 4))) #We have 4 input features
model.add(Dense(10, activation='sigmoid', batch_input_shape=(1,)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='sigmoid'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Compile the model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=opt,
              metrics=['accuracy'])

model.summary()